In [1]:
import pandas as pd
from lib.pitch_control import plot_pitch_control, KNNPitchControl, SpearmanPitchControl, FernandezPitchControl
from lib.draw import pitch
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/all_goals.csv', low_memory=False)
df.edgecolor.fillna('white', inplace=True)
df.bgcolor.fillna('black', inplace=True)
df.sample(5)

,bgcolor,dx,dy,edgecolor,frame,play,player,player_num,team,x,y,z,coords,player_obj,num,name,play_id
65552,darkred,-0.126536,1.687730e-02,black,26,Bournemouth 0 - 3 Liverpool,12087,NaN,defense,48.909321,58.892057,0.0,NaN,NaN,NaN,NaN,2
54708,blue,0.002541,-1.614015e-03,white,48,Liverpool [1] - 0 Everton,2606,NaN,defense,10.307798,45.490381,0.0,NaN,NaN,NaN,NaN,6
73880,red,0.000000,-2.131628e-14,white,69,Leicester 0 - [3] Liverpool,8101,NaN,attack,55.782313,36.974790,0.0,NaN,NaN,NaN,NaN,5
25850,red,0.008527,-3.144893e-03,white,132,Liverpool [4] - 0 Barcelona,747,66.0,attack,98.696338,0.308924,0.0,NaN,NaN,NaN,NaN,17
63738,red,-0.115260,9.718962e-03,white,154,Liverpool [2] - 0 Everton,42401,NaN,attack,68.114305,47.859294,0.0,NaN,NaN,NaN,NaN,9


In [3]:
df.play.unique()

array(['Liverpool [3] - 0 Bournemouth', 'Bayern 0 - [1] Liverpool',
       'Fulham 0 - [1] Liverpool', 'Southampton 1 - [2] Liverpool',
       'Liverpool [2] - 0 Porto', 'Porto 0 - [2] Liverpool',
       'Liverpool [4] - 0 Barcelona', 'Liverpool [1] - 0 Wolves',
       'Liverpool [3] - 0 Norwich', 'Liverpool [2] - 1 Chelsea',
       'Liverpool [2] - 1 Newcastle', 'Liverpool [2] - 0 Salzburg',
       'Genk 0 - [3] Liverpool', 'Liverpool [2] - 0 Man City',
       'Liverpool [1] - 0 Everton', 'Liverpool [2] - 0 Everton',
       'Bournemouth 0 - 3 Liverpool', 'Liverpool [1] - 0 Watford',
       'Leicester 0 - [3] Liverpool', 'Barcelona 1 - [2] Real Madrid'],
      dtype=object)

In [4]:
knn_pitch_control_voronoi = KNNPitchControl()
knn_pitch_control_spearman = KNNPitchControl(lags=[0, 10, 20], smoothing=8)
knn_pitch_control_fernandez = KNNPitchControl(lags=[0, 10, 20], smoothing=6, distance_basis=250)
spearman_pitch_control = SpearmanPitchControl()
fernandez_pitch_control = FernandezPitchControl()

In [5]:
# Initialize results file
import os
results_file_path = 'res/results.csv'
grids_file_path = 'res/grids.csv'
if os.path.isfile(results_file_path):
    df_results = pd.read_csv(results_file_path)
    df_grids = pd.read_csv(grids_file_path)
else:
    df_results = pd.DataFrame(columns=['play_frame_id'])
    df_grids = pd.DataFrame(columns=['play_frame_id', 'model_name'] + [f'c{i}' for i in range(106*69)])

In [6]:
def run_model(df_results, df_grids, model, model_name, play_frame_id, df_frame, df_play=None):
    t0 = time()
    if df_play is None:
        model.grid['control'] = model.predict(df_frame).round(2)
    else:
        model.grid['control'] = model.predict(df_frame, df_play).round(2)
    inference_time = time() - t0
    
    model.grid['play_frame_id'] = play_frame_id
    model.grid['model'] = model_name
    
    df_results = pd.concat([df_results, 
        pd.DataFrame([[play_frame_id, model_name, play, frame_no, inference_time]], 
        columns=['play_frame_id', 'model', 'play', 'frame', 'inference_time'])])[['play_frame_id', 'model', 'play', 'frame', 'inference_time']]
    df_grids.loc[len(df_grids)] = [play_frame_id, model_name] + model.grid['control'].tolist()

    return df_results, df_grids

In [7]:
play_frame_id = 0
for play in df.play.unique()[[4, 5, -1]]:
    print(play)
    df_play = df[df.play == play]
    for frame_no in tqdm(df_play.frame.unique()):
        play_frame_id += 1
        if play_frame_id in df_results.play_frame_id.unique():
            continue

        df_frame = df_play[df_play.frame == frame_no]
        
        df_results, df_grids = run_model(df_results, df_grids, knn_pitch_control_voronoi, 'KNN (Voronoi)', play_frame_id, df_frame)
        df_results, df_grids = run_model(df_results, df_grids, knn_pitch_control_spearman, 'KNN (Spearman)', play_frame_id, df_frame)
        df_results, df_grids = run_model(df_results, df_grids, knn_pitch_control_fernandez, 'KNN (Fernandez)', play_frame_id, df_frame)
        df_results, df_grids = run_model(df_results, df_grids, spearman_pitch_control, 'Spearman Pitch Control', play_frame_id, df_frame)
        df_results, df_grids = run_model(df_results, df_grids, fernandez_pitch_control, 'Fernandez Pitch Control', play_frame_id, df_frame, df_play)

        df_results.to_csv(results_file_path, index=False)
        df_grids.to_csv(grids_file_path, index=False)

  0%|          | 0/195 [00:00<?, ?it/s]

Liverpool [2] - 0 Porto


 16%|█▌        | 31/195 [15:31<1:29:04, 32.59s/it]